Ein Testdatensatz wird erstellt um den OCR-Prozess zu überprüfen.

In [1]:
import pandas as pd

df = pd.read_csv("../all_csv_images/all_data_all_images_no_duplicates.csv")
test_df = df.sample(n=50)
test_df.to_csv('../all_csv_images/test_file.csv', index=False)


Es wurden zwei Methoden ausprobiert um OCR durchzuführen. Als erstes wurde pytessereact in Verbindung mit dem Python Paket PIL ausprobiert. Jedoch waren die Ergebnisse nicht zufriedenstellend, da bei Text der nicht schwarz auf weiß gestanden ist, nicht erkannt wurde. Als zweites wurde die Google Cloud Vision API verwendet. Diese hat eine bessere Erkennungsrate und ist auch für große Mengen an Bildern geeignet. Aus diesem Grund haben wir uns für diese entschieden.

Bei der ersten Methode werden die Bilder vor der Verarbeitung von pytesseract mithilfe von PIL vorverarbeitet. Die Bilder werden in Graustufen umgewandelt und daraufhin wird der Kontrast verbessert um mit Pytesseract OCR durchzuführen.

In [2]:
import pandas as pd
import pytesseract
from PIL import Image
from tqdm import tqdm
from PIL import Image, ImageEnhance


pytesseract.pytesseract.tesseract_cmd = (
    r"C:\Program Files\Tesseract-OCR\tesseract.exe"
)


df = pd.read_csv('../all_csv_images/test_file.csv')

df['ocr_text'] = ''

with open("ocr_log.txt", "w") as log:
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Performing OCR'):
        try:
            image = Image.open(row['relative_path'])
            image = image.convert("L")
            contrast = ImageEnhance.Contrast(image)
            image = contrast.enhance(2)
            ocr_text = pytesseract.image_to_string(image)
            df.at[index, 'ocr_text'] = ocr_text
        except Exception as e:
            log.write(f'Error processing image {row["relative_path"]}: {e}\n')

df.to_csv('../all_csv_images/ocr_PIL_enhance.csv', index=False)


Performing OCR: 100%|██████████| 50/50 [00:20<00:00,  2.39it/s]


Bei diesem Beispiel wurde versucht mit cv2 anstatt mit PIL die Bilder vorzuverarbeiten, jedoch waren die Ergebnisse sehr ähnlich.

In [26]:
import pandas as pd
import pytesseract
from tqdm import tqdm
import logging
import cv2

pytesseract.pytesseract.tesseract_cmd = (
    r"C:\Program Files\Tesseract-OCR\tesseract.exe"
)

logging.basicConfig(filename='ocr.log', level=logging.DEBUG)

df = pd.read_csv('../all_csv_images/test_file.csv')

df['ocr_text'] = ''

with open("ocr_log.txt", "w") as log:
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Performing OCR'):
        try:
            image = cv2.imread(row['relative_path'])
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 3)
            gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            ocr_text = pytesseract.image_to_string(gray)
            df.at[index, 'ocr_text'] = ocr_text
        except Exception as e:
            log.write(f'Error processing image {row["relative_path"]}: {e}\n')

df.to_csv('../all_csv_images/ocr_cv2.csv', index=False)

Performing OCR: 100%|██████████| 50/50 [00:16<00:00,  3.02it/s]


Quelle des Codes: https://cloud.google.com/vision/docs/ocr?hl=de

Mit der Google Vision API können die Bilder ohne Vorverarbeitung an die API geschickt werden. Der erkannte Text wird in einem JSON-Format zurückgegeben. Aus diesen JSON Dateien wird der Text in einer CSV Datei gespeichert.

In [29]:
import pandas as pd
import io
from google.cloud import vision
from google.oauth2 import service_account
from tqdm import tqdm


df = pd.read_csv('../all_csv_images/all_data_all_images_no_duplicates.csv')


df['ocr_text'] = ''
# Die Credentials wurden hier gelöscht, da sie nicht öffentlich sein sollten
credentials = service_account.Credentials.from_service_account_file('')

client = vision.ImageAnnotatorClient(credentials=credentials)

with open("ocr_log.txt", "w") as log:
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Performing OCR'):
        try:
            with io.open(row['relative_path'], 'rb') as image_file:
                content = image_file.read()
            image = vision.Image(content=content)
            response = client.text_detection(image=image)
            ocr_text = response.text_annotations
            df.at[index, 'ocr_text'] = ocr_text[0].description
        except Exception as e:
            log.write(f'Error processing image {row["relative_path"]}: {e}\n')


    df.to_csv('../all_csv_images/ocr_google_vision.csv', index=False)


Performing OCR: 100%|██████████| 28052/28052 [6:08:50<00:00,  1.27it/s]   
